In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from openslide import OpenSlide
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import os
import time

csv_path = "/kaggle/input/prostate-cancer-grade-assessment/train.csv"
df = pd.read_csv(csv_path)
total_images = len(df)

# File to store assessments
assessment_csv_path = "image_assessments.csv"
image_assessments = {}
image_timestamps = {}

# Current user
current_user = "HyphenSaad"

# Load existing assessments if file exists
if os.path.exists(assessment_csv_path):
    assessment_df = pd.read_csv(assessment_csv_path)
    for _, row in assessment_df.iterrows():
        image_assessments[row['image_id']] = row['assessment']
        image_timestamps[row['image_id']] = row['timestamp']
else:
    # Create a new assessments file with headers
    assessment_df = pd.DataFrame(columns=['image_id', 'assessment', 'timestamp', 'username'])
    assessment_df.to_csv(assessment_csv_path, index=False)

def get_current_timestamp():
    current_time = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime())
    dt = datetime.strptime(current_time, "%Y-%m-%d %H:%M:%S")
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def save_to_csv(image_id, assessment):
    global assessment_df
    timestamp = get_current_timestamp()
    image_timestamps[image_id] = timestamp
    
    # Check if the image_id already exists in the DataFrame
    if os.path.exists(assessment_csv_path):
        assessment_df = pd.read_csv(assessment_csv_path)
        
        if image_id in assessment_df['image_id'].values:
            # Update existing record
            assessment_df.loc[assessment_df['image_id'] == image_id, 'assessment'] = assessment
            assessment_df.loc[assessment_df['image_id'] == image_id, 'timestamp'] = timestamp
            assessment_df.loc[assessment_df['image_id'] == image_id, 'username'] = current_user
        else:
            # Append new record
            new_row = pd.DataFrame({'image_id': [image_id], 'assessment': [assessment], 
                                  'timestamp': [timestamp], 'username': [current_user]})
            assessment_df = pd.concat([assessment_df, new_row], ignore_index=True)
    else:
        # Create new DataFrame with this record
        assessment_df = pd.DataFrame({'image_id': [image_id], 'assessment': [assessment], 
                                    'timestamp': [timestamp], 'username': [current_user]})
    
    # Save to CSV
    assessment_df.to_csv(assessment_csv_path, index=False)
    
    return timestamp

def display_image(index, output_widget):
    row = df.iloc[index]
    image_id = row['image_id']
    data_provider = row['data_provider']
    isup_grade = row['isup_grade']
    gleason_score = row['gleason_score']
    
    image_path = f"/kaggle/input/prostate-cancer-grade-assessment/train_images/{image_id}.tiff"
    
    try:
        with output_widget:
            clear_output(wait=True)
            
            slide = OpenSlide(image_path)
            level = min(2, slide.level_count - 1)
            level_dim = slide.level_dimensions[level]
            level_img = slide.read_region((0, 0), level, level_dim)
            level_img = np.array(level_img.convert('RGB'))
            
            fig = plt.figure(figsize=(16, 9))
            ax = plt.subplot(111)
            ax.imshow(level_img)
            ax.set_title(f"Image {index+1}/{total_images} - ID: {image_id} (Level {level})", fontsize=14)
            ax.set_aspect('equal')
            ax.axis('off')
            plt.tight_layout()
            plt.subplots_adjust(left=0, right=1, top=0.95, bottom=0)
            plt.show()
            
            slide.close()
            
    except Exception as e:
        with output_widget:
            clear_output(wait=True)
            plt.figure(figsize=(16, 9))
            plt.text(0.5, 0.5, f"Error loading image {image_id}: {str(e)}", 
                    ha='center', va='center', fontsize=14)
            plt.axis('off')
            plt.show()
    
    return {
        'index': index,
        'image_id': image_id,
        'data_provider': data_provider,
        'isup_grade': isup_grade,
        'gleason_score': gleason_score
    }

def update_metadata(meta_widget, image_info):
    with meta_widget:
        clear_output(wait=True)
        print(f"Image: {image_info['index']+1}/{total_images}")
        print(f"Image ID: {image_info['image_id']}")
        print(f"Data Provider: {image_info['data_provider']}")
        print(f"ISUP Grade: {image_info['isup_grade']}")
        print(f"Gleason Score: {image_info['gleason_score']}")
        
        image_id = image_info['image_id']
        if image_id in image_assessments:
            print(f"Assessment: {image_assessments[image_id]}")
            
            if image_id in image_timestamps:
                print(f"Last assessed: {image_timestamps[image_id]}")
                
        current_time = get_current_timestamp()
        print(f"\nTimestamp (UTC): {current_time}")
        print(f"User: {current_user}")

def interactive_widget_viewer():
    image_output = widgets.Output()
    meta_output = widgets.Output()
    
    current_index = [0]
    current_image_info = [None]
    
    assessment_radio = widgets.RadioButtons(
        options=['No Issue', 'Black Area', 'Ink Spill'],
        value='No Issue',
        description='Assessment:',
        disabled=False,
        layout=widgets.Layout(margin='10px 0')
    )
    
    def update_display(index):
        image_info = display_image(index, image_output)
        current_image_info[0] = image_info
        
        if image_info['image_id'] in image_assessments:
            assessment_radio.value = image_assessments[image_info['image_id']]
        else:
            assessment_radio.value = 'No Issue'
            
        update_metadata(meta_output, image_info)
        current_index[0] = index
    
    def save_assessment(image_id, assessment):
        image_assessments[image_id] = assessment
        save_to_csv(image_id, assessment)
        update_metadata(meta_output, current_image_info[0])
    
    assessment_radio.observe(lambda change: save_assessment(
        current_image_info[0]['image_id'], change['new']
    ) if change['name'] == 'value' and current_image_info[0] else None, names='value')
    
    assessment_status = widgets.HTML(
        value=f"<div style='font-size: 14px; margin: 10px 0;'><b>Assessment Status</b></div>"
    )
    
    csv_status = widgets.HTML(
        value=f"<p style='font-size: 14px;'>CSV: {assessment_csv_path}</p>"
    )
    
    assessment_count = widgets.HTML(
        value=f"<p style='font-size: 14px;'>Assessments: {len(image_assessments)}/{total_images}</p>"
    )
    
    prev_button = widgets.Button(
        description='Previous',
        disabled=False,
        button_style='info',
        tooltip='Go to previous image',
        icon='arrow-left',
        layout=widgets.Layout(width='45%', height='40px')
    )
    
    next_button = widgets.Button(
        description='Next',
        disabled=False,
        button_style='info',
        tooltip='Go to next image',
        icon='arrow-right',
        layout=widgets.Layout(width='45%', height='40px')
    )
    
    index_input = widgets.IntText(
        value=1,
        description='Go to:',
        min=1,
        max=total_images,
        step=1,
        layout=widgets.Layout(width='60%', height='36px')
    )
    
    go_button = widgets.Button(
        description='Go',
        disabled=False,
        button_style='success',
        tooltip='Go to specified index',
        icon='check',
        layout=widgets.Layout(width='30%', height='36px')
    )
    
    status_label = widgets.HTML(
        value=f"<p style='font-size: 14px; margin-top:10px;'>Total images: {total_images}</p>"
    )
    
    def update_assessment_count():
        assessment_count.value = f"<p style='font-size: 14px;'>Assessments: {len(image_assessments)}/{total_images}</p>"
    
    def on_prev_button_clicked(b):
        if current_image_info[0]:
            save_assessment(current_image_info[0]['image_id'], assessment_radio.value)
        
        update_assessment_count()    
        new_index = max(current_index[0] - 1, 0)
        index_input.value = new_index + 1
        update_display(new_index)
    
    def on_next_button_clicked(b):
        if current_image_info[0]:
            save_assessment(current_image_info[0]['image_id'], assessment_radio.value)
        
        update_assessment_count()
        new_index = min(current_index[0] + 1, total_images - 1)
        index_input.value = new_index + 1
        update_display(new_index)
    
    def on_go_button_clicked(b):
        if current_image_info[0]:
            save_assessment(current_image_info[0]['image_id'], assessment_radio.value)
        
        update_assessment_count()
        new_index = index_input.value - 1
        if 0 <= new_index < total_images:
            update_display(new_index)
        else:
            with meta_output:
                clear_output(wait=True)
                print(f"Index must be between 1 and {total_images}")
    
    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)
    go_button.on_click(on_go_button_clicked)
    
    nav_controls1 = widgets.HBox([prev_button, next_button])
    nav_controls2 = widgets.HBox([index_input, go_button])
    
    left_panel = widgets.VBox([
        widgets.HTML(value="<h3 style='font-size: 16px;'>Controls</h3>"),
        nav_controls1, 
        nav_controls2,
        assessment_radio,
        assessment_status,
        assessment_count,
        csv_status,
        widgets.HTML(value="<h3 style='font-size: 16px;'>Metadata</h3>"),
        meta_output
    ], layout=widgets.Layout(
        width='30%', 
        margin='10px', 
        padding='15px',
        background_color='#f0f0f0',
        border='1px solid #ddd'
    ))
    
    right_panel = widgets.VBox([
        image_output
    ], layout=widgets.Layout(
        width='70%'
    ))
    
    main_layout = widgets.HBox([left_panel, right_panel])
    
    update_display(0)
    update_assessment_count()
    
    display(main_layout)

interactive_widget_viewer()